<a href="https://colab.research.google.com/github/arnabbiswas66/multimodal-fake-news-classifier/blob/main/Twitter_text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U "tensorflow-text==2.11.*"

In [2]:
!pip install -q tf-models-official==2.11.0

In [3]:

import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os
import re
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
tf.get_logger().setLevel('ERROR')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
def get_df(file):
    return pd.read_csv(file,sep = '\t')

In [6]:
train_df = get_df('/content/drive/MyDrive/multimodal-news/mediaeval2015/devset/tweets.txt')
test_df = get_df('/content/drive/MyDrive/multimodal-news/mediaeval2015/testset/tweets.txt')

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14277 entries, 0 to 14276
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweetId     14277 non-null  int64 
 1   tweetText   14277 non-null  object
 2   userId      14277 non-null  int64 
 3   imageId(s)  14277 non-null  object
 4   username    14277 non-null  object
 5   timestamp   14277 non-null  object
 6   label       14277 non-null  object
dtypes: int64(2), object(5)
memory usage: 780.9+ KB


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweetId     3755 non-null   int64 
 1   tweetText   3755 non-null   object
 2   userId      3755 non-null   int64 
 3   imageId(s)  3755 non-null   object
 4   username    3755 non-null   object
 5   timestamp   3755 non-null   object
 6   label       3755 non-null   object
dtypes: int64(2), object(5)
memory usage: 205.5+ KB


In [9]:
#Changing 'humor' to 'fake'
train_df.loc[(train_df.label == 'humor'),'label'] = 'fake'
test_df.loc[(test_df.label == 'humor'),'label'] = 'fake'

In [13]:
#Removing retweets, reposts, and modified tweets
rtPattern1 = "(RT|rt|MT|mt|RP|rp):? @\w*:?"
rtPattern2 = "(\bRT\b|\brt\b|\bMT\b|\bmt\b|\bRP\b|\brp\b)"
rtPattern3 = "(@\w*:)"
rtPattern4 = "(#rt|#RT|#mt|#MT|#rp|#retweet|#Retweet|#modifiedtweet|#modifiedTweet|#ModifiedTweet|#repost|#Repost)"
rtPattern5 = "(via @\w*)"

retweets = train_df['tweetText'].str.contains(rtPattern1)
train_df = train_df[~retweets]

retweets = train_df['tweetText'].str.contains(rtPattern2)
train_df = train_df[~retweets]

retweets = train_df['tweetText'].str.contains(rtPattern3)
train_df = train_df[~retweets]

retweets = train_df['tweetText'].str.contains(rtPattern4)
train_df = train_df[~retweets]

retweets = train_df['tweetText'].str.contains(rtPattern5)
train_df = train_df[~retweets]

train_df.reset_index(drop=True, inplace=True)
train_df.shape

<ipython-input-13-fdd562f6ac27>:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retweets = train_df['tweetText'].str.contains(rtPattern1)
<ipython-input-13-fdd562f6ac27>:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retweets = train_df['tweetText'].str.contains(rtPattern2)
<ipython-input-13-fdd562f6ac27>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retweets = train_df['tweetText'].str.contains(rtPattern3)
<ipython-input-13-fdd562f6ac27>:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retweets = train_df['tweetText'].str.contains(rtPattern4)
<ipython-input-13-fdd562f6ac27>:20: UserWarning: This pattern is interpreted as a reg

(11550, 7)

In [10]:

#Removing remaining twitter handles @username
train_df['tweetText'] = train_df['tweetText'].apply(lambda text: re.sub(r'@\w*', "", text))

In [11]:
#Removing emojis
emojis = re.compile("["
                    u"\U0001F600-\U0001F64F"
                    u"\U0001F300-\U0001F5FF"
                    u"\U0001F680-\U0001F6FF"
                    u"\U0001F1E0-\U0001F1FF"
                    u"\U00002702-\U000027B0"
                    u"\U000024C2-\U0001F251"
                    "]+", flags=re.UNICODE)

train_df['tweetText'] = train_df['tweetText'].apply(lambda text: emojis.sub(r'', text) if emojis.search(text) else text)

In [12]:
#Cleaning symbols - ampersand and newline
train_df['tweetText'] = train_df['tweetText'].apply(lambda text: re.sub(r'&amp;|\\n', '', text))

In [13]:
#Removing urls
train_df['tweetText'] = train_df['tweetText'].apply(lambda text: re.sub(r'http\S+', '', text))
train_df['tweetText'] = train_df['tweetText'].apply(lambda text: re.sub(r'\\\/\S+', '', text))

In [14]:
#Removing whitespace
train_df['tweetText'] = train_df['tweetText'].apply(lambda text: " ".join(text.split()))

In [19]:
#Initialise stopwords

stopwords = nltk.corpus.stopwords.words()
stopwords.extend([':', ';', '[', ']', '"', "'", '(', ')', '.', '?', '#', '@', '...'])

In [20]:
#Removing stopwords
train_df['filteredTweet'] = train_df['tweetText'].apply(lambda x: ' '.join([w for w in x.split() if w not in stopwords]))

In [15]:
train_df['label'].unique()

array(['fake', 'real'], dtype=object)

In [16]:
train_df['label'] = train_df['label'].apply(lambda x: 0 if x=='fake' else 1)

In [17]:
train_df['label'].unique()

array([0, 1])

In [18]:
final_train_df = train_df[['tweetText', 'label']]
final_train_df.head()

,tweetText,label
0,¿Se acuerdan de la película: “El día después d...,0
1,: Miren a Sandy en NY! Tremenda imagen del hur...,0
2,"Buena la foto del Huracán Sandy, me recuerda a...",0
3,Scary shit #hurricane #NY,0
4,My fave place in the world #nyc #hurricane #sa...,0


In [19]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", name='preprocessing')
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=False, name='encoder')

In [20]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text-layer')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
d_layer = tf.keras.layers.Dropout(0.1, name="dropout-layer")(outputs['pooled_output'])
d_layer = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(d_layer)
model = tf.keras.Model(inputs=[text_input], outputs = [d_layer])

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text-layer (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_word_ids':   0           ['text-layer[0][0]']             
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [22]:
m= [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

In [23]:
# Optimizer
epochs = 10
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=m)

In [24]:
X_train = final_train_df['tweetText']
y_train = final_train_df['label']

In [25]:
model.fit(X_train, y_train, epochs=epochs, batch_size=32)

Epoch 1/10
447/447 [==============================] - 161s 335ms/step - loss: 0.6403 - accuracy: 0.6443 - precision: 0.4233 - recall: 0.0880
Epoch 2/10
447/447 [==============================] - 151s 338ms/step - loss: 0.6215 - accuracy: 0.6634 - precision: 0.5361 - recall: 0.1735
Epoch 3/10
447/447 [==============================] - 151s 339ms/step - loss: 0.6047 - accuracy: 0.6779 - precision: 0.5931 - recall: 0.2091
Epoch 4/10
447/447 [==============================] - 151s 338ms/step - loss: 0.5954 - accuracy: 0.6919 - precision: 0.6244 - recall: 0.2662
Epoch 5/10
447/447 [==============================] - 151s 338ms/step - loss: 0.5903 - accuracy: 0.6957 - precision: 0.6326 - recall: 0.2796
Epoch 6/10
447/447 [==============================] - 151s 337ms/step - loss: 0.5840 - accuracy: 0.7052 - precision: 0.6552 - recall: 0.3054
Epoch 7/10
447/447 [==============================] - 151s 337ms/step - loss: 0.5828 - accuracy: 0.7049 - precision: 0.6468 - recall: 0.3170
Epoch 8/10
44